***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@stat.sinica.edu.tw </p>
### <p> Institute of Statistical Science, Academia Sinica, Taipei, Taiwan.</p>  
---

# Input

In [30]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime

In [31]:
# Input
MNIST     = True
TRIALS    = 1
timestr   = ''
#NUM_NEI  = 4 # Others is 4
#NUM_NEI   = 2 # MNIST is 5
#Region_Index_loc = 6 # column location; Others is 6
REGION_INDEX_LOC = 4 # column location; MNIST is 4

PATH1='../phase3_data/MNIST_seedinds.txt'
PATH2='../phase3_data/MNIST_bilabels.txt'
PATH3='../phase3_data/MNIST_seedinds_neighborregions.txt'
PATH4='../phase3_data/MNIST_Labels_Spec20.csv'
PATH5='../phase3_data/Small_MNIST_tSNE_embeddings.pickle'
#PATH5='../phase3_data/MNIST_Labels_5000.csv'
#=================================================================

# (1)PATH1
df = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df = pandas.read_csv(PATH4)
#1213 add auto judge
all_region_index = df.to_numpy().T[REGION_INDEX_LOC].astype(int)

region: [15  9 11  6  7  8 10 20 14 12 19  1 13 18  4]
region amount: 15


# Get neighbors

In [32]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  15
[[ 9]
 [15]
 [16]
 [ 7]
 [ 6]
 [ 8]
 [20]
 [10]
 [ 5]
 [ 6]
 [ 1]
 [19]
 [13]
 [ 4]
 [18]]
neighbor amount:  14


[[9],
 [15],
 [16],
 [7],
 [6],
 [8],
 [20],
 [10],
 [5],
 [],
 [1],
 [19],
 [13],
 [4],
 [18]]

In [33]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[15, 9],
 [9, 15],
 [11, 16],
 [6, 7],
 [7, 6],
 [8, 8],
 [10, 20],
 [20, 10],
 [14, 5],
 [12],
 [19, 1],
 [1, 19],
 [13, 13],
 [18, 4],
 [4, 18]]

In [34]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../phase3_data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

** original duplicate at i= 5 ;region 8 ;duplicate size 491
** same duplicate situation
** original duplicate at i= 12 ;region 13 ;duplicate size 460
** same duplicate situation


In [37]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (5000, 3)
training size: 7151


# CNN original

In [38]:
#======================================================

#1213 add auto judge
from CNN_Modules_1D import ME_CNN

# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

n, p1, p2 0 0 0
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_5 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_6 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_3 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 84

6435/6435 [==============================] - 0s 35us/step - loss: 1.4622 - accuracy: 0.3228 - val_loss: 1.4661 - val_accuracy: 0.2877
Epoch 48/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.4644 - accuracy: 0.3203 - val_loss: 1.4889 - val_accuracy: 0.3073
Epoch 49/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4626 - accuracy: 0.3305 - val_loss: 1.4841 - val_accuracy: 0.3254
Epoch 50/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4593 - accuracy: 0.3235 - val_loss: 1.4837 - val_accuracy: 0.2919
Epoch 51/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.4596 - accuracy: 0.3267 - val_loss: 1.4780 - val_accuracy: 0.3115
Epoch 52/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4652 - accuracy: 0.3200 - val_loss: 1.4823 - val_accuracy: 0.3115
Epoch 53/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.4573 - accuracy: 0.3294 - val_loss: 1.4810 - val_accurac

6435/6435 [==============================] - 0s 37us/step - loss: 1.6424 - accuracy: 0.3001 - val_loss: 1.5913 - val_accuracy: 0.3212
Epoch 9/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.6257 - accuracy: 0.3032 - val_loss: 1.5725 - val_accuracy: 0.3115
Epoch 10/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.6043 - accuracy: 0.2974 - val_loss: 1.5548 - val_accuracy: 0.3017
Epoch 11/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.5903 - accuracy: 0.3072 - val_loss: 1.5424 - val_accuracy: 0.3226
Epoch 12/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.5781 - accuracy: 0.3077 - val_loss: 1.5303 - val_accuracy: 0.3282
Epoch 13/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.5711 - accuracy: 0.3040 - val_loss: 1.5176 - val_accuracy: 0.3394
Epoch 14/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.5580 - accuracy: 0.3083 - val_loss: 1.5393 - val_accuracy

6435/6435 [==============================] - 0s 38us/step - loss: 1.2811 - accuracy: 0.3994 - val_loss: 1.2552 - val_accuracy: 0.4092
Epoch 65/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.2844 - accuracy: 0.3949 - val_loss: 1.2670 - val_accuracy: 0.3980
Epoch 66/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.2869 - accuracy: 0.3855 - val_loss: 1.2412 - val_accuracy: 0.4022
Epoch 67/80
6435/6435 [==============================] - 0s 33us/step - loss: 1.2768 - accuracy: 0.3911 - val_loss: 1.2390 - val_accuracy: 0.4050
Epoch 68/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.2737 - accuracy: 0.4045 - val_loss: 1.2463 - val_accuracy: 0.4022
Epoch 69/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.2751 - accuracy: 0.3946 - val_loss: 1.2440 - val_accuracy: 0.4106
Epoch 70/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.2747 - accuracy: 0.3977 - val_loss: 1.2431 - val_accurac

6435/6435 [==============================] - 0s 34us/step - loss: 1.4805 - accuracy: 0.3082 - val_loss: 1.4886 - val_accuracy: 0.2919
Epoch 26/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4781 - accuracy: 0.3226 - val_loss: 1.4694 - val_accuracy: 0.3380
Epoch 27/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4752 - accuracy: 0.3144 - val_loss: 1.4650 - val_accuracy: 0.3310
Epoch 28/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.4831 - accuracy: 0.3158 - val_loss: 1.4698 - val_accuracy: 0.3561
Epoch 29/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4774 - accuracy: 0.3172 - val_loss: 1.4865 - val_accuracy: 0.3198
Epoch 30/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4823 - accuracy: 0.3197 - val_loss: 1.4669 - val_accuracy: 0.3547
Epoch 31/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4766 - accuracy: 0.3119 - val_loss: 1.4692 - val_accurac

[[1.0936469e-01 1.2382034e-01 2.3591762e-13 ... 3.9329389e-01
  1.4913493e-01 1.9631407e-01]
 [3.7018657e-02 3.4828149e-02 1.5820419e-19 ... 8.9257339e-04
  5.0201081e-03 4.0380638e-03]
 [2.1925037e-01 2.4744773e-01 1.1435459e-15 ... 7.8666329e-02
  2.0135823e-01 2.4868277e-01]
 ...
 [2.4656411e-01 2.5993302e-01 1.1577379e-16 ... 9.5530525e-03
  2.1019526e-01 2.7273774e-01]
 [1.3392623e-01 1.3965544e-01 1.8493547e-12 ... 1.7892893e-01
  2.1927890e-01 2.9470643e-01]
 [1.3327296e-01 1.4606556e-01 5.7430796e-13 ... 2.0061362e-01
  2.1480753e-01 2.8158915e-01]]
[12 10 14 ... 14 14 14]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:01:03.803814
n, p1, p2 3 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 

6435/6435 [==============================] - 0s 37us/step - loss: 1.3031 - accuracy: 0.3868 - val_loss: 1.3218 - val_accuracy: 0.3813
Epoch 43/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.2989 - accuracy: 0.3854 - val_loss: 1.3180 - val_accuracy: 0.3603
Epoch 44/80
6435/6435 [==============================] - 0s 48us/step - loss: 1.3065 - accuracy: 0.3860 - val_loss: 1.3175 - val_accuracy: 0.3478
Epoch 45/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.2963 - accuracy: 0.3877 - val_loss: 1.3112 - val_accuracy: 0.3561
Epoch 46/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.2854 - accuracy: 0.3879 - val_loss: 1.3064 - val_accuracy: 0.3883
Epoch 47/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.2812 - accuracy: 0.3913 - val_loss: 1.3109 - val_accuracy: 0.3687
Epoch 48/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.2834 - accuracy: 0.3983 - val_loss: 1.3055 - val_accurac

6435/6435 [==============================] - 0s 41us/step - loss: 2.0335 - accuracy: 0.2493 - val_loss: 2.0285 - val_accuracy: 0.2821
Epoch 4/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.9094 - accuracy: 0.2721 - val_loss: 1.9438 - val_accuracy: 0.2891
Epoch 5/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.8283 - accuracy: 0.2831 - val_loss: 1.8668 - val_accuracy: 0.2570
Epoch 6/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.7571 - accuracy: 0.2945 - val_loss: 1.7986 - val_accuracy: 0.2765
Epoch 7/80
6435/6435 [==============================] - 0s 34us/step - loss: 1.7082 - accuracy: 0.2948 - val_loss: 1.7332 - val_accuracy: 0.2779
Epoch 8/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.6676 - accuracy: 0.3082 - val_loss: 1.6966 - val_accuracy: 0.2696
Epoch 9/80
6435/6435 [==============================] - 0s 43us/step - loss: 1.6434 - accuracy: 0.3030 - val_loss: 1.6912 - val_accuracy: 0.2

6435/6435 [==============================] - 0s 38us/step - loss: 1.4117 - accuracy: 0.3472 - val_loss: 1.4429 - val_accuracy: 0.3254
Epoch 60/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4040 - accuracy: 0.3576 - val_loss: 1.4518 - val_accuracy: 0.3128
Epoch 61/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.3986 - accuracy: 0.3605 - val_loss: 1.4345 - val_accuracy: 0.3282
Epoch 62/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.3917 - accuracy: 0.3579 - val_loss: 1.4296 - val_accuracy: 0.3212
Epoch 63/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.3890 - accuracy: 0.3699 - val_loss: 1.4193 - val_accuracy: 0.3436
Epoch 64/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.3844 - accuracy: 0.3812 - val_loss: 1.4129 - val_accuracy: 0.3701
Epoch 65/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.3717 - accuracy: 0.3700 - val_loss: 1.4179 - val_accurac

In [40]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

(5, 5000) (5, 5000, 15)


# CNN combination and removal

In [ ]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

105

n, p1, p2 0 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84

6435/6435 [==============================] - 0s 40us/step - loss: 1.4164 - accuracy: 0.3653 - val_loss: 1.4330 - val_accuracy: 0.3771
Epoch 48/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.3970 - accuracy: 0.3706 - val_loss: 1.4057 - val_accuracy: 0.3645
Epoch 49/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.3718 - accuracy: 0.3796 - val_loss: 1.3664 - val_accuracy: 0.3841
Epoch 50/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.3432 - accuracy: 0.3828 - val_loss: 1.3723 - val_accuracy: 0.3855
Epoch 51/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.3303 - accuracy: 0.3792 - val_loss: 1.3353 - val_accuracy: 0.3953
Epoch 52/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.3151 - accuracy: 0.3817 - val_loss: 1.3207 - val_accuracy: 0.3939
Epoch 53/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.3065 - accuracy: 0.3843 - val_loss: 1.3329 - val_accurac

6435/6435 [==============================] - 0s 39us/step - loss: 1.4558 - accuracy: 0.3497 - val_loss: 1.4576 - val_accuracy: 0.3589
Epoch 9/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4395 - accuracy: 0.3493 - val_loss: 1.4416 - val_accuracy: 0.3645
Epoch 10/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4301 - accuracy: 0.3506 - val_loss: 1.4360 - val_accuracy: 0.3659
Epoch 11/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4238 - accuracy: 0.3500 - val_loss: 1.4317 - val_accuracy: 0.3589
Epoch 12/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4157 - accuracy: 0.3556 - val_loss: 1.4274 - val_accuracy: 0.3603
Epoch 13/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4118 - accuracy: 0.3538 - val_loss: 1.4246 - val_accuracy: 0.3743
Epoch 14/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4060 - accuracy: 0.3521 - val_loss: 1.4063 - val_accuracy

6435/6435 [==============================] - 0s 38us/step - loss: 1.2046 - accuracy: 0.4180 - val_loss: 1.2228 - val_accuracy: 0.4511
Epoch 65/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.2098 - accuracy: 0.4180 - val_loss: 1.2086 - val_accuracy: 0.4274
Epoch 66/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.1981 - accuracy: 0.4266 - val_loss: 1.1987 - val_accuracy: 0.4372
Epoch 67/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.1953 - accuracy: 0.4185 - val_loss: 1.2044 - val_accuracy: 0.4162
Epoch 68/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.1925 - accuracy: 0.4183 - val_loss: 1.1863 - val_accuracy: 0.4385
Epoch 69/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.1885 - accuracy: 0.4219 - val_loss: 1.2094 - val_accuracy: 0.4176
Epoch 70/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.1889 - accuracy: 0.4159 - val_loss: 1.1987 - val_accurac

...

Epoch 57/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.4816 - accuracy: 0.3166 - val_loss: 1.4686 - val_accuracy: 0.2989
Epoch 58/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.4813 - accuracy: 0.3144 - val_loss: 1.4533 - val_accuracy: 0.3380
Epoch 59/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4776 - accuracy: 0.3240 - val_loss: 1.4577 - val_accuracy: 0.3282
Epoch 60/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4787 - accuracy: 0.3221 - val_loss: 1.4572 - val_accuracy: 0.3184
Epoch 61/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4760 - accuracy: 0.3179 - val_loss: 1.4506 - val_accuracy: 0.3422
Epoch 62/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4739 - accuracy: 0.3229 - val_loss: 1.4563 - val_accuracy: 0.3268
Epoch 63/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4775 - accuracy: 0.3235 - val_loss: 1.4698 -

6435/6435 [==============================] - 0s 39us/step - loss: 1.4776 - accuracy: 0.3256 - val_loss: 1.5074 - val_accuracy: 0.3520
Epoch 34/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4750 - accuracy: 0.3218 - val_loss: 1.5173 - val_accuracy: 0.3520
Epoch 35/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.4724 - accuracy: 0.3239 - val_loss: 1.5117 - val_accuracy: 0.3408
Epoch 36/80
6435/6435 [==============================] - 0s 42us/step - loss: 1.4697 - accuracy: 0.3352 - val_loss: 1.4989 - val_accuracy: 0.3659
Epoch 37/80
6435/6435 [==============================] - 0s 38us/step - loss: 1.4667 - accuracy: 0.3324 - val_loss: 1.5015 - val_accuracy: 0.3366
Epoch 38/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.4609 - accuracy: 0.3335 - val_loss: 1.4931 - val_accuracy: 0.3534
Epoch 39/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.4579 - accuracy: 0.3302 - val_loss: 1.4977 - val_accurac

Train on 6435 samples, validate on 716 samples
Epoch 1/80
6435/6435 [==============================] - 1s 80us/step - loss: 2.6093 - accuracy: 0.1709 - val_loss: 2.5324 - val_accuracy: 0.1606
Epoch 2/80
6435/6435 [==============================] - 0s 43us/step - loss: 2.3075 - accuracy: 0.1821 - val_loss: 2.0789 - val_accuracy: 0.2346
Epoch 3/80
6435/6435 [==============================] - 0s 44us/step - loss: 1.9685 - accuracy: 0.2465 - val_loss: 1.8941 - val_accuracy: 0.3073
Epoch 4/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.8305 - accuracy: 0.2768 - val_loss: 1.7767 - val_accuracy: 0.3310
Epoch 5/80
6435/6435 [==============================] - 0s 37us/step - loss: 1.7512 - accuracy: 0.2786 - val_loss: 1.7286 - val_accuracy: 0.3226
Epoch 6/80
6435/6435 [==============================] - 0s 40us/step - loss: 1.6878 - accuracy: 0.2959 - val_loss: 1.6455 - val_accuracy: 0.3436
Epoch 7/80
6435/6435 [==============================] - 0s 35us/step - loss: 1.6423

Epoch 57/80
6435/6435 [==============================] - 0s 34us/step - loss: 1.4102 - accuracy: 0.3582 - val_loss: 1.4082 - val_accuracy: 0.3687
Epoch 58/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.3992 - accuracy: 0.3571 - val_loss: 1.3791 - val_accuracy: 0.3757
Epoch 59/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.3959 - accuracy: 0.3669 - val_loss: 1.3779 - val_accuracy: 0.3813
Epoch 60/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.3813 - accuracy: 0.3695 - val_loss: 1.3952 - val_accuracy: 0.3771
Epoch 61/80
6435/6435 [==============================] - 0s 36us/step - loss: 1.3714 - accuracy: 0.3731 - val_loss: 1.3560 - val_accuracy: 0.3897
Epoch 62/80
6435/6435 [==============================] - 0s 34us/step - loss: 1.3622 - accuracy: 0.3737 - val_loss: 1.3410 - val_accuracy: 0.3869
Epoch 63/80
6435/6435 [==============================] - 0s 32us/step - loss: 1.3529 - accuracy: 0.3837 - val_loss: 1.3418 -

6435/6435 [==============================] - 0s 33us/step - loss: 1.4200 - accuracy: 0.3576 - val_loss: 1.4079 - val_accuracy: 0.3631
Epoch 19/80
6435/6435 [==============================] - 0s 34us/step - loss: 1.4155 - accuracy: 0.3514 - val_loss: 1.4212 - val_accuracy: 0.3492
Epoch 20/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4072 - accuracy: 0.3607 - val_loss: 1.4040 - val_accuracy: 0.3561
Epoch 21/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4043 - accuracy: 0.3591 - val_loss: 1.4175 - val_accuracy: 0.3394
Epoch 22/80
6435/6435 [==============================] - 0s 39us/step - loss: 1.4029 - accuracy: 0.3576 - val_loss: 1.4022 - val_accuracy: 0.3184
Epoch 23/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.3968 - accuracy: 0.3610 - val_loss: 1.4043 - val_accuracy: 0.3464
Epoch 24/80
6435/6435 [==============================] - 0s 41us/step - loss: 1.4007 - accuracy: 0.3571 - val_loss: 1.3880 - val_accurac

In [ ]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

In [ ]:
print("done")